In [1]:
import numpy as np

def rnn_forward(inputs, h0, W_xh, W_hh, W_hy, b_h, b_y):
    """
    Forward pass for a simple RNN.

    Parameters
    ----------
    inputs : list or array of shape (T, input_dim)
        Sequence of input vectors.
    h0 : array of shape (hidden_dim,)
        Initial hidden state.
    W_xh : array of shape (hidden_dim, input_dim)
    W_hh : array of shape (hidden_dim, hidden_dim)
    W_hy : array of shape (output_dim, hidden_dim)
    b_h : array of shape (hidden_dim,)
    b_y : array of shape (output_dim,)

    Returns
    -------
    hs : list of hidden states (length T+1, hs[0] = h0)
    ys : list of outputs (length T)
    caches : list of dict, each dict has intermediate values needed for BPTT
    """
    T = len(inputs)
    hs = [h0]
    ys = []
    caches = []

    for t in range(T):
        x_t = inputs[t]

        # Pre-activation
        z_t = np.dot(W_xh, x_t) + np.dot(W_hh, hs[-1]) + b_h
        # Hidden state
        h_t = np.tanh(z_t)
        # Output
        y_t = np.dot(W_hy, h_t) + b_y

        hs.append(h_t)
        ys.append(y_t)

        # Store values needed for backprop
        cache = {
            "x_t": x_t,       # current input
            "z_t": z_t,       # pre-activation of hidden
            "h_t": h_t,       # current hidden state
            "h_prev": hs[-2], # previous hidden state
        }
        caches.append(cache)

    return hs, ys, caches

def mse_loss(ys, targets):
    """
    Compute the Mean Squared Error (MSE) loss over T time steps.

    ys : list of predicted outputs (each shape = (output_dim,))
    targets : list of target outputs (same shape as ys)

    Returns
    -------
    loss : float
    """
    T = len(ys)
    loss = 0.0
    for t in range(T):
        diff = ys[t] - targets[t]
        # 0.5 * sum of squares is convenient for derivatives
        loss += 0.5 * np.sum(diff**2)
    return loss

def rnn_backward(ys, targets, hs, caches, W_xh, W_hh, W_hy, b_h, b_y):
    """
    Backpropagation Through Time for a simple RNN using MSE loss.

    Parameters
    ----------
    ys : list of predicted outputs (length T)
    targets : list of target outputs (length T)
    hs : list of hidden states (length T+1, hs[0] is h0)
    caches : list of dict, each with
        'x_t', 'z_t', 'h_t', 'h_prev'
    W_xh, W_hh, W_hy, b_h, b_y : RNN parameters

    Returns
    -------
    grads : dict of gradients w.r.t. each parameter
    """
    # Initialize gradients to zero
    dW_xh = np.zeros_like(W_xh)
    dW_hh = np.zeros_like(W_hh)
    dW_hy = np.zeros_like(W_hy)
    db_h  = np.zeros_like(b_h)
    db_y  = np.zeros_like(b_y)

    # We'll need the gradient wrt the hidden state for each time
    T = len(ys)
    dh_next = np.zeros_like(hs[0])  # shape = (hidden_dim,)

    # Backprop through time (from the last time step to the first)
    for t in reversed(range(T)):
        # 1) Gradient of loss wrt y_t
        dy_t = (ys[t] - targets[t])  # shape = (output_dim,)

        # 2) Grad wrt W_hy, b_y
        dW_hy += np.outer(dy_t, hs[t+1])  # hs[t+1] = h_t
        db_y  += dy_t

        # 3) Grad wrt h_t, add gradient from future hidden states (dh_next)
        dh_t = np.dot(W_hy.T, dy_t) + dh_next  # shape = (hidden_dim,)

        # 4) Grad wrt z_t (where z_t is pre-activation)
        # h_t = tanh(z_t) => d h_t / d z_t = 1 - h_t^2
        h_t = hs[t+1]
        dz_t = dh_t * (1 - h_t**2)

        # 5) Grad wrt W_xh, W_hh, b_h
        x_t = caches[t]["x_t"]
        h_prev = caches[t]["h_prev"]

        dW_xh += np.outer(dz_t, x_t)
        dW_hh += np.outer(dz_t, h_prev)
        db_h  += dz_t

        # 6) Grad wrt h_{t-1} -> used for next iteration
        dh_next = np.dot(W_hh.T, dz_t)

    # Compile gradients in a dictionary
    grads = {
        "W_xh": dW_xh,
        "W_hh": dW_hh,
        "W_hy": dW_hy,
        "b_h": db_h,
        "b_y": db_y
    }
    return grads


if __name__ == "__main__":
    # -------------------------------
    # 1. Hyperparameters
    # -------------------------------
    input_dim = 3
    hidden_dim = 4
    output_dim = 2
    T = 5             # sequence length
    lr = 0.01         # learning rate

    # -------------------------------
    # 2. Create synthetic data
    # -------------------------------
    np.random.seed(42)
    inputs = [np.random.randn(input_dim) for _ in range(T)]
    targets = [np.random.randn(output_dim) for _ in range(T)]

    # -------------------------------
    # 3. Initialize parameters
    # -------------------------------
    W_xh = np.random.randn(hidden_dim, input_dim) * 0.1
    W_hh = np.random.randn(hidden_dim, hidden_dim) * 0.1
    W_hy = np.random.randn(output_dim, hidden_dim) * 0.1
    b_h  = np.zeros(hidden_dim)
    b_y  = np.zeros(output_dim)

    # Initial hidden state
    h0 = np.zeros(hidden_dim)

    # -------------------------------
    # 4. Forward pass
    # -------------------------------
    hs, ys, caches = rnn_forward(inputs, h0, W_xh, W_hh, W_hy, b_h, b_y)

    # -------------------------------
    # 5. Compute loss
    # -------------------------------
    loss_value = mse_loss(ys, targets)

    # -------------------------------
    # 6. Backward pass (BPTT)
    # -------------------------------
    grads = rnn_backward(ys, targets, hs, caches, W_xh, W_hh, W_hy, b_h, b_y)

    # -------------------------------
    # 7. Parameter update
    # -------------------------------
    W_xh -= lr * grads["W_xh"]
    W_hh -= lr * grads["W_hh"]
    W_hy -= lr * grads["W_hy"]
    b_h  -= lr * grads["b_h"]
    b_y  -= lr * grads["b_y"]

    # -------------------------------
    # 8. Print results
    # -------------------------------
    print("Loss before update:", loss_value)
    print("Grad W_xh:\n", grads["W_xh"])
    print("Grad W_hh:\n", grads["W_hh"])
    print("Grad W_hy:\n", grads["W_hy"])
    print("Grad b_h:\n", grads["b_h"])
    print("Grad b_y:\n", grads["b_y"])


Loss before update: 4.42928666078666
Grad W_xh:
 [[ 0.14190064 -0.03085566 -0.16590029]
 [ 0.2048493  -0.25288768 -0.26299734]
 [ 0.22665322 -0.39487468 -0.23698683]
 [-0.19705645  0.26551573  0.22413435]]
Grad W_hh:
 [[ 0.00652463 -0.00843032  0.05760394  0.0185553 ]
 [ 0.01580351 -0.00631523  0.06006751  0.02529782]
 [ 0.0139037  -0.00806739  0.04922337  0.01920442]
 [-0.01248669  0.00766473 -0.05047733 -0.01995666]]
Grad W_hy:
 [[ 0.15304402  0.00765917  0.83909957  0.28807495]
 [ 0.26431672  0.09696962 -0.08838056  0.30032227]]
Grad b_h:
 [ 0.16823351  0.3340134   0.42787586 -0.32430673]
Grad b_y:
 [3.37718054 1.00718345]
